In [1]:
import os
import psycopg2
from psycopg2.extras import DictCursor
import pandas as pd

In [2]:
# Function that connects to database
def db_connect():
    db_name = os.environ['db_name']
    db_user = os.environ['db_user']
    db_host = os.environ['db_host']
    db_credentials = os.environ['db_creds']
  
    conn_string = "dbname='" + str(db_name) + "' user='" + str(db_user) + "' host='" + str(db_host) + "' password='" + str(db_credentials) + "'"

    try:
        conn = psycopg2.connect(str(conn_string))
        conn.autocommit = True
    except:
        print("Unable to connect to the database")

    cur = conn.cursor(cursor_factory=DictCursor)
    return cur

cur = db_connect()

In [3]:
# Load weather data to DataFrame
query = "SELECT * FROM weather.weather_clean"
cur.execute(query)
results = cur.fetchall()

flat_results = []
for result in results:
    flat_results.append(result[0])
weather_df = pd.DataFrame(flat_results)
weather_df.head()

,0
0,1990-01-01
1,1990-01-02
2,1990-01-03
3,1990-01-04
4,1990-01-05


In [4]:
# Load disaster data to DataFrame
query = "SELECT * FROM weather.disasters_clean"
cur.execute(query)
results = cur.fetchall()

flat_results = []
for result in results:
    flat_results.append(result[0])
disaster_df = pd.DataFrame(flat_results)
disaster_df.head()

,0
0,1900-0006-JAM
1,1900-0008-JPN
2,1900-0003-USA
3,1900-0007-JAM
4,1900-9001-IND
